# CipherRecognizer
#### Jan Jasiński & Jan Wilczek

###### Technologia Mowy AGH 2017

### Wstęp

Celem projektu "CipherRecognizer" jest stworzenie prostego systemu rozpoznawania nagranych nazw cyfr zaimplementowany w języku Python.
Link do repozytorium: https://github.com/Jasinsk/CipherRecognizer

### Raport 1 (16.11.17-23.11.17)

1. Założenia

   Do naszego projektu wybraliśmy parametryzację MFCC sygnału mowy - do jej wyznaczania używamy biblioteki python_speech_features.

   Jako klasyfikatora chcemy użyć odpowiednio wyszkolonej sieci neuronowej. Zdecydowaliśmy się na klasyfikator MLPClassifier z biblioteki sklearn.neural_network. Na jego właściwym ustawieniu opierać się będzie znaczna część procesu dopasowania modelu nauki klasyfikatora.
   
   Wejściem sieci neuronowej będzie superwektor o długości 13 + 13x13 = 182, złożony z uśrednionego po czasie wektora parametrów MFCC wraz z kolejnymi rzędami macierzy kowariancji ustawionymi jeden za drugim w ciąg.
   
1. Architektura systemu

   Cały nasz projekt opiera się na klasach. Obecnie ich lista wraz z metodami wygląda następująco:
   
   1. `GlobalParameters` (niezaimplementowana)
     - klasa w modelu singleton zawierająca wszystkie parametry, którymi można regulować (dostrajać) system.
   1. `WaveFile` - klasa odpowiedzialna za odczytanie danych z pliku wave.
    * `__init__(self, filepath)`  - konstruktor, filepath określa ścieżkę dostępu do danego pliku z nagraniem cyfry.
    * `data(self, normalize=True)` - zwraca dane z pliku audio jako listę z wartościami typu float w zakresie <-1,1>.
   1. `MFCCParametrizer` - klasa odpowiedzialna za parametryzację MFCC sygnału.
    * `__init__(self,winlen=0.025,winstep=0.01,numcep=13,nfilt=26,nfft=512,preemph=0.97,ceplifter=22,               appendEnergy=True)` - konstruktor ustawiający parametry wyznaczania MFCC zgodnie z kolejnymi argumentami.
    * `parameters(self, signal, samplerate)` - zwraca macierz wyznaczonych parametrów z podanego sygnału.
	* `super_vector(self, signal, samplerate)` - zwraca superwektor parametrów wyznaczonych na podstawie danego sygnału.
   1. `ANNClassifier` - klasa będąca opakowaniem dla sieci neuronowej.
    * `__init__(self, nb_hidden_layers, nb_neurons_in_layer, activation_function='relu', solver='lbfgs', nb_iterations=200)` - konstruktor klasyfikatora z zadanymi parametrami. W przyszłości zmienimy nieco tę implementację, żeby w każdej warstwie sieci nie było tyle samo neuronów.
    * `train(self, training_input_data, training_output_data)` - wytrenowuje model podając kolejne wektory z wejściami i odpowiadające im wyjścia.
    * `predict(self, test_input_data)` - oblicza logarytmiczne prawdopodobieństwa każdej klasy (w naszym przypadku jest 10 klas odpowiadających 10 cyfrom) i zwraca ich wektor.
    
   1. `ResultHolder` - klasa odpowiadająca za zbieranie wyników (prawdopodobieństw) w testach kros-walidacji i obliczania skuteczności dla zadanych parametrów. Rezultaty można następnie eksportować do pliku .xls.
    * `__init__(self, classes)` - konstruktor przekazujący nazwy klas. Przekazywanym wektorem klas powinien być `ANNClassifier.MLPClassifier.classes_`.
    * `add_result(self, prediction_vector, correct_result)` - dodaje kolejne wyniki otrzymywane w wyniku wywołania metody `predict()` klasy `ANNClassifier` i zapisuje wraz z przekazanym poprawnym wynikiem.
    * `error_rate(self)`- oblicza procent błędów wszystkich dodanych wyników.
    * `write_results_to_excel_file(self, filename, sheet)` - wypisuje wszystkie wyniki wraz z ich poprawnymi odpowiedziami, poprawnością (za odpowiedź uznaje się klasę z największym prawdopodobieństwem) oraz końcowy procent błędów do pliku .xls.
    * `__is_correct(self, index)` - prywatna metoda zwracająca wartość logiczną mówiącą o tym, czy odpowiedź wyniku o podanym indeksie jest poprawna.
  
   1. `Recognizer` (niezaimplementowana) - główna klasa odpowiedzialna za pętle testów, zmianę parametrów itp. Jej metody nie są jeszcze sprecyzowane.

   1. `ConfigurationManager` (niezaimplementowana) - klasa służąca do generowania konfiguracji w sposób deterministyczny (dla przykładu: jeśli podamy liczbę plików testowych równą 10 każde wywołanie metody `__generate_configurations` powinno wygenerować te same konfiguracje).
    * `__init__(self, foldername)` - konstruktor z przekazywaną nazwą folderu z plikami audio.
    * `__generate_configurations(self, nb_test_files)` - generuje konfiguracje do kros-walidacji o zadanej liczbie testów w każdym zbiorze.
    * `nb_configurations(self)` - zwraca liczbę konfiguracji.
    * `test_data(self, configuration_id)` - zwraca ścieżki plików wybrane do testu w konfiguracji o danym indeksie.
    * `training_data(self, configuration_id)` - zwraca ścieżki plików wybrane do treningu w konfiguracji o danym indeksie.
    * `output_configurations_to_file(self, filename)` - wypisuje konfiguracje do pliku.
   
1. Postęp prac
 
    Udało nam się do tej pory zaimplementować wstępnie klasy ANNClassifier, MFCCParametrizer, ResultHandler i WavFile wraz z testami. Klasy te są gotowe do zastosowania we wstępnej implementacji pętli treningowo-testowej służącej do wybrania odpowiednich parametrów systemu. Największą trudnością będzie dobór wielkości i liczby warstw w sieci neuronowej, ponieważ brak nam doświadczenia w tej dziedzinie.

   


